In [2]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', 50)

In [3]:
#comm_17 = pd.read_excel('../data/commodity/p16021coll2_3014.xlsx')
#comm_00_16 = pd.read_excel('../data/commodity/p16021coll2_1796.xlsx')
#lock_data = pd.read_csv('../data/commodity/lock_codes.csv', dtype={'Lock No':str})
#lock_info = lock_info[lock_info['River Code'] == 'OH']

comm_00_17 = pd.read_csv('../data/commodity/joined_p16021_comm_oh_2000_2017.csv')
comm_00_17['ShortTons'] = comm_00_17['ShortTons'].str.replace(',','').astype(float)

In [4]:
comm_by_year = pd.pivot_table(comm_00_17, values='ShortTons', index='CommodityName', columns=['CompletedYear'], aggfunc=np.sum).reset_index()
comm_by_year.columns = comm_by_year.columns.astype(str)
comm_by_year.columns

Index(['CommodityName', '2000', '2001', '2002', '2003', '2004', '2005', '2006',
       '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015',
       '2016', '2017'],
      dtype='object', name='CompletedYear')

In [5]:
total_tons_by_year = comm_00_17.groupby('CompletedYear')['ShortTons'].sum().reset_index()
#total_tons_by_year.to_csv('../data/commodity/analyzed/comm-2000-2017-by-year.csv')

It looks like Ohio River commodity shipments have decreased ~24% between 2000 and 2017. Now that's a short ton analysis so it's somewhat possibly that there are still the same number of shipments/barges going... and just what they're shipping is lighter.

I think what you need to do is calculate the 2000-2017 diff for each `CommodityName` and see which have decreased the most. I mean... even if the heaviest things have decreased the most, you really can't make deductions here... you'd really need to see *barges* by `CommodityName` by year. Maybe talk to Kenney about that... or see if there is any kind of conversion you can do that would translate short tons to barges.

Also you really need to make sure the analysis you're doing isn't duplicating shipments. You're summing the `ShortTon` column so you need to make sure none of the other columns make that dangerously inaccurate.

In [6]:
comm_by_year

CompletedYear,CommodityName,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Acyclic Hydrocarbons,91064.0,79348.0,90102.0,85228.0,72294.0,83358.0,100897.0,95742.0,77921.0,36938.0,67247.0,53784.0,52870.0,67962.0,66200.0,63004.0,35362.0,46259.0
1,Alcoholic Beverages,7349.0,7007.0,5722.0,8462.0,9812.0,8465.0,9917.0,13826.0,11102.0,15371.0,12903.0,16023.0,12524.0,13078.0,19214.0,22835.0,23412.0,26999.0
2,Alcohols,1478979.0,1288697.0,1370066.0,1513507.0,1513779.0,1442555.0,1423355.0,1400374.0,1369641.0,1275529.0,1415602.0,1520690.0,1444094.0,1255325.0,1331523.0,1463910.0,1479124.0,1367179.0
3,Aluminum,420550.0,177833.0,451911.0,263525.0,376073.0,575326.0,313953.0,286340.0,168417.0,139717.0,227767.0,299308.0,263805.0,162199.0,150044.0,156821.0,283540.0,531720.0
4,Aluminum Ore,2475353.0,2573759.0,2640740.0,2682453.0,2392028.0,2116052.0,2134411.0,2417004.0,2750923.0,1968077.0,2020263.0,2205706.0,2294807.0,2118438.0,1833693.0,1661759.0,1002033.0,991632.0
5,Ammonia,288705.0,276619.0,202178.0,265084.0,312538.0,322109.0,304601.0,362273.0,271239.0,293817.0,267509.0,341318.0,301440.0,327234.0,318237.0,235957.0,174628.0,222213.0
6,"Animal Feed, Prep.",2613430.0,2479067.0,2314808.0,1951016.0,1904229.0,1693678.0,1749017.0,1468625.0,1270590.0,975588.0,1360572.0,1292561.0,1218696.0,1024499.0,923962.0,1129359.0,920512.0,1402491.0
7,Animals & Prod. NEC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3200.0,NaN,1700.0,NaN
8,"Asphalt, Tar & Pitch",2843447.0,2333500.0,2494540.0,2319807.0,2718033.0,2684734.0,2568647.0,1751759.0,2139909.0,1119721.0,1248941.0,1111486.0,1100203.0,734242.0,1003385.0,1150175.0,521201.0,585914.0
9,Bananas & Plantains,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3200.0,NaN,NaN,NaN


Ok... so apparently we just don't ship Nitrogen Func. Comp. anymore. I don't know what those are. Let's find out.

Also, I think what we need to do is calculate the annual decrease in `CommodityName` shipping and then take the average between 2000 and 2017. Because maybe there's just like... mad variability in shipping.

In [14]:
## 2000 - 2017 difference by CommodityName
keep_cols = ['2000','2001','2002','2003','2004','2005',
             '2006','2007','2008','2009','2010','2011',
             '2012','2013','2014','2015','2016','2017']
comm_diff = comm_by_year.pct_change(axis='columns', fill_method=None)
comm_diff = comm_diff[keep_cols]

comm_diff['avg_2000_2017'] = comm_diff.mean(axis=1, numeric_only=True, skipna=True)
comm_diff['median_2000_2017'] = comm_diff.median(axis=1, numeric_only=True, skipna=True)
comm_diff['diff_2000_2017'] = (comm_by_year['2017'] - comm_by_year['2000'])/ comm_by_year['2000']

#comm_diff.columns
join_comm = comm_by_year[['CommodityName']]
comm_diff = comm_diff.join(join_comm, lsuffix='_l', rsuffix='_r')

#comm_diff.sort_values('diff_2000_2017', ascending=False)
comm_diff = comm_diff[['CommodityName','avg_2000_2017','2000','median_2000_2017','diff_2000_2017',
                       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008','2009',
                       '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017']]

comm_diff

CompletedYear,CommodityName,avg_2000_2017,2000,median_2000_2017,diff_2000_2017,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
0,Acyclic Hydrocarbons,0.005018,NaN,-0.037102,-0.492017,-0.128657,0.135530,-0.054094,-0.151758,0.153042,0.210406,-0.051092,-0.186136,-0.525956,0.820537,-0.200202,-0.016994,0.285455,-0.025926,-0.048278,-0.438734,0.308156
1,Alcoholic Beverages,0.103977,NaN,0.128595,2.673833,-0.046537,-0.183388,0.478854,0.159537,-0.137281,0.171530,0.394172,-0.197020,0.384525,-0.160562,0.241804,-0.218374,0.044235,0.469185,0.188456,0.025268,0.153212
2,Alcohols,-0.001765,NaN,-0.007537,-0.075593,-0.128658,0.063141,0.104696,0.000180,-0.047050,-0.013310,-0.016146,-0.021946,-0.068713,0.109816,0.074236,-0.050369,-0.130718,0.060700,0.099425,0.010393,-0.075683
3,Aluminum,0.145513,NaN,-0.014886,0.264344,-0.577142,1.541210,-0.416865,0.427087,0.529825,-0.454304,-0.087953,-0.411829,-0.170410,0.630202,0.314097,-0.118617,-0.385156,-0.074939,0.045167,0.808049,0.875291
4,Aluminum Ore,-0.041243,NaN,-0.000852,-0.599398,0.039754,0.026025,0.015796,-0.108268,-0.115373,0.008676,0.132399,0.138154,-0.284576,0.026516,0.091792,0.040396,-0.076856,-0.134413,-0.093764,-0.397005,-0.010380
5,Ammonia,0.003434,NaN,-0.012030,-0.230311,-0.041863,-0.269110,0.311142,0.179015,0.030623,-0.054354,0.189336,-0.251286,0.083240,-0.089539,0.275912,-0.116835,0.085569,-0.027494,-0.258549,-0.259916,0.272494
6,"Animal Feed, Prep.",-0.018416,NaN,-0.061702,-0.463352,-0.051413,-0.066258,-0.157159,-0.023981,-0.110570,0.032674,-0.160314,-0.134844,-0.232177,0.394617,-0.049987,-0.057146,-0.159348,-0.098133,0.222300,-0.184925,0.523599
7,Animals & Prod. NEC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"Asphalt, Tar & Pitch",-0.052039,NaN,-0.027745,-0.793942,-0.179341,0.069012,-0.070046,0.171663,-0.012251,-0.043240,-0.318023,0.221577,-0.476744,0.115404,-0.110057,-0.010151,-0.332630,0.366559,0.146295,-0.546851,0.124161
9,Bananas & Plantains,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Of the things that the Ohio River ships frequently\*, here are the things that they have been shipping less of over the past 18 years\**:
- I&S Plates & Sheets (-6.65%)
- Iron Ore (-5.20%)
- Asphalt, Tar & Pitch (-5.20%)
- Fab. Metal Products (-4.51%)
- Lime (-4.30%)
- Aluminum Ore (-4.12%)
- Other Hydrocarbons (-3.24%)
- Coal & Lignite (-2.29%) 
- Residual Fuel Oil (-2.12%)
- Gasoline (-2.06%)
- Animal Feed, Prep. (-1.84%)
- Cement & Concrete (-1.06%)
- Sodium Hydroxide (-0.36%)
- Sand & Gravel (-0.22%)
- Alcohols (-0.18%)

_NOTE: Number in paraenthesis is an average of annual short ton shipping differences for all years betwee 2000 and 2017._

To be clear, when I say that we have been shipping less of these things, I'm talking about average shipping between years over the past 18 years (2000 - 2017). I'm actually also talking about the % difference between 2000 and 2017. These things _do_ seem a bit variable year to year but... I'm pretty confident saying that the Ohio River is shipping less of this shit now than we were in 2000.


\* Frequently means that it has shipping tonnage records for each year between years 2000 - 2017

\** (2000 - 2017)